## General Processing

Workflow:
1. General filter
2. Outliers detection
3. Train and test set splitting
4. Missing values imputation
5. Add Score values
6. Output the train and test data




### Load master dataset

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
from importlib import reload
from helpers import *
import dataset_path
# from sklearn.impute import SimpleImputer
reload(dataset_path)
from dataset_path import output_path, mimic_iv_path
path = output_path
print(path)
df_master = pd.read_csv(os.path.join(path, 'master_dataset.csv'))

2022-08-19 19:07:48.218194: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


/gpfs/scratch/ag7948/mimic/mimic-iv-1.0/output


In [30]:
from sklearn.impute import SimpleImputer

In [3]:
print(df_master.shape)
print(df_master.columns.tolist())

(448972, 119)
['index', 'subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'anchor_age', 'gender', 'anchor_year', 'dod', 'admittime', 'dischtime', 'deathtime', 'ethnicity', 'edregtime', 'edouttime', 'insurance', 'in_year', 'age', 'outcome_inhospital_mortality', 'ed_death', 'before_ed_mortality', 'ed_los', 'intime_icu', 'time_to_icu_transfer', 'outcome_icu_transfer_12h', 'outcome_hospitalization', 'outcome_critical', 'n_ed_30d', 'n_ed_90d', 'n_ed_365d', 'next_ed_visit_time', 'next_ed_visit_time_diff', 'outcome_ed_revisit_3d', 'n_hosp_30d', 'n_hosp_90d', 'n_hosp_365d', 'n_icu_30d', 'n_icu_90d', 'n_icu_365d', 'ed_los_hours', 'time_to_icu_transfer_hours', 'next_ed_visit_time_diff_days', 'triage_temperature', 'triage_heartrate', 'triage_resprate', 'triage_o2sat', 'triage_sbp', 'triage_dbp', 'triage_pain', 'triage_acuity', 'chiefcomplaint', 'chiefcom_chest_pain', 'chiefcom_abdominal_pain', 'chiefcom_headache', 'chiefcom_shortness_of_breath', 'chiefcom_back_pain', 'chiefcom_cough', 'chi

In [4]:
# pd.set_option('display.max_columns', 200) 
df_master.head()

,index,subject_id,hadm_id,stay_id,intime,outtime,anchor_age,gender,anchor_year,dod,...,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon
0,335314,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,52,F,2180,NaN,...,0,97.7,79.0,16.0,98.0,107.0,60.0,0.0,0.0,9.0
1,335315,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,52,F,2180,NaN,...,1,97.9,86.0,17.0,93.0,96.0,57.0,5.0,3.0,12.0
2,335316,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,52,F,2180,NaN,...,1,98.2,85.0,18.0,98.0,81.0,38.0,0.0,2.0,14.0
3,335317,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,52,F,2180,NaN,...,1,99.0,96.0,18.0,97.0,86.0,45.0,NaN,8.0,14.0
4,335318,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,52,F,2180,NaN,...,1,98.1,91.0,18.0,99.0,98.0,60.0,NaN,6.0,7.0


### 1. General filter - Age, triage_acuity

In [5]:
df_master = df_master[df_master['age'] >= 18]

In [6]:
# na_master = df_master[df_master['triage_acuity'].isnull()]
# len(na_master)

In [7]:
df_master = df_master[df_master['triage_acuity'].notnull()]
len(df_master)

441437

### 2. Outlier Detection 

In [8]:
# from mimic-extract
vitals_valid_range = {
    'temperature': {'outlier_low': 14.2, 'valid_low': 26, 'valid_high': 45, 'outlier_high':47},
    'heartrate': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 350, 'outlier_high':390},
    'resprate': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 300, 'outlier_high':330},
    'o2sat': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 100, 'outlier_high':150},
    'sbp': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 375, 'outlier_high':375},
    'dbp': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 375, 'outlier_high':375},
    'pain': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 10, 'outlier_high':10},
    'acuity': {'outlier_low': 1, 'valid_low': 1, 'valid_high': 5, 'outlier_high':5},
}

In [9]:
df_master = convert_temp_to_celcius(df_master)

In [10]:
display_outliers_count(df_master, vitals_valid_range)

,variable,< outlier_low,"[outlier_low, valid_low)","[valid_low, valid_high]","(valid_high, outlier_high]",> outlier_high
0,triage_temperature,512,29,423329,0,10
1,triage_heartrate,0,0,430515,0,7
2,triage_resprate,0,0,427096,0,4
3,triage_o2sat,0,0,426773,6,43
4,triage_sbp,0,0,429234,0,19
5,triage_dbp,0,0,427984,0,423
6,triage_pain,2,0,406760,0,11860
7,triage_acuity,0,0,441437,0,0
8,ed_temperature_last,675,43,417317,1,48
9,ed_heartrate_last,0,0,425769,0,0


In [11]:
df_master = remove_outliers(df_master, vitals_valid_range)

In [12]:
len(df_master)

441437

In [15]:
type(df_master)

pandas.core.frame.DataFrame

### 3. Dataset Split (train:0.7, validation:0.1, test: 0.2, use seed to fix)

In [21]:
# df_train = df_master.sample(frac=0.7,random_state=10) #random state is a seed value
# df_test = df_master.drop(df_train.index)
df_train, df_validation, df_test = np.split(df_master.sample(frac=1, random_state=42), [int(.7*len(df_master)), int(.8*len(df_master))])

In [22]:
type(df_train)

pandas.core.frame.DataFrame

In [23]:
len(df_train)

309005

In [24]:
len(df_validation)

44144

In [25]:
len(df_test)

88288

In [26]:
df_train.head()

,index,subject_id,hadm_id,stay_id,intime,outtime,anchor_age,gender,anchor_year,dod,...,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon
374508,47399,18336789,NaN,33947176,2204-07-21 18:27:00,2204-07-21 18:37:00,71,F,2197,NaN,...,0,37.111111,92.0,18.0,98.0,146.0,84.0,0.0,0.0,15.0
165548,187780,13712250,23685749.0,37806132,2185-08-06 23:42:00,2185-08-08 14:09:00,67,F,2176,NaN,...,1,36.722222,51.0,16.0,98.0,115.0,66.0,8.0,12.0,11.0
324893,432843,17236922,NaN,30242272,2165-02-28 04:37:00,2165-02-28 04:54:00,63,M,2163,NaN,...,0,36.722222,82.0,18.0,96.0,137.0,90.0,0.0,0.0,4.0
226240,307634,15057835,28433076.0,34192717,2172-05-05 03:39:00,2172-05-05 05:46:00,68,M,2171,NaN,...,0,36.833333,56.0,19.0,99.0,148.0,78.0,0.0,2.0,2.0
349811,210916,17800278,NaN,35819741,2165-09-26 17:30:00,2165-09-26 17:36:00,76,M,2158,NaN,...,1,37.000000,80.0,16.0,100.0,156.0,98.0,NaN,0.0,0.0


### 4. Missing Value imputation 

In [27]:
df_missing_stats = df_train.isnull().sum().to_frame().T
df_missing_stats.loc[1] = df_missing_stats.loc[0] / len(df_train)
df_missing_stats.index = ['no. of missing values', 'percentage of missing values']
df_missing_stats

,index,subject_id,hadm_id,stay_id,intime,outtime,anchor_age,gender,anchor_year,dod,...,eci_Depression,ed_temperature_last,ed_heartrate_last,ed_resprate_last,ed_o2sat_last,ed_sbp_last,ed_dbp_last,ed_pain_last,n_med,n_medrecon
no. of missing values,0.0,0.0,162875.000000,0.0,0.0,0.0,0.0,0.0,0.0,296952.000000,...,0.0,16954.000000,11063.000000,11416.000000,18859.000000,11229.000000,11411.000000,33159.000000,0.0,0.0
percentage of missing values,0.0,0.0,0.527095,0.0,0.0,0.0,0.0,0.0,0.0,0.960994,...,0.0,0.054866,0.035802,0.036944,0.061031,0.036339,0.036928,0.107309,0.0,0.0


In [28]:
vitals_cols = [col for col in df_master.columns if len(col.split('_')) > 1 and 
                                                   col.split('_')[1] in vitals_valid_range and
                                                   col.split('_')[1] != 'acuity']
vitals_cols

['triage_temperature',
 'triage_heartrate',
 'triage_resprate',
 'triage_o2sat',
 'triage_sbp',
 'triage_dbp',
 'triage_pain',
 'ed_temperature_last',
 'ed_heartrate_last',
 'ed_resprate_last',
 'ed_o2sat_last',
 'ed_sbp_last',
 'ed_dbp_last',
 'ed_pain_last']

In [31]:
imputer = SimpleImputer(strategy='median')
df_train[vitals_cols] = imputer.fit_transform(df_train[vitals_cols])
df_validation[vitals_cols] = imputer.transform(df_validation[vitals_cols])
df_test[vitals_cols] = imputer.transform(df_test[vitals_cols])

### 5. Add Score values

In [32]:
# train
add_triage_MAP(df_train) # add an extra variable MAP
add_score_CCI(df_train)
add_score_CART(df_train)
add_score_REMS(df_train)
add_score_NEWS(df_train)
add_score_NEWS2(df_train)
add_score_MEWS(df_train)
# validation
add_triage_MAP(df_validation) # add an extra variable MAP
add_score_CCI(df_validation)
add_score_CART(df_validation)
add_score_REMS(df_validation)
add_score_NEWS(df_validation)
add_score_NEWS2(df_validation)
add_score_MEWS(df_validation)
# test
add_triage_MAP(df_test) # add an extra variable MAP
add_score_CCI(df_test)
add_score_CART(df_test)
add_score_REMS(df_test)
add_score_NEWS(df_test)
add_score_NEWS2(df_test)
add_score_MEWS(df_test)

Variable 'add_triage_MAP' successfully added
Variable 'add_score_CCI' successfully added
Variable 'Score_CART' successfully added
Variable 'Score_REMS' successfully added
Variable 'Score_NEWS' successfully added
Variable 'Score_NEWS2' successfully added
Variable 'Score_MEWS' successfully added
Variable 'add_triage_MAP' successfully added
Variable 'add_score_CCI' successfully added
Variable 'Score_CART' successfully added
Variable 'Score_REMS' successfully added
Variable 'Score_NEWS' successfully added
Variable 'Score_NEWS2' successfully added
Variable 'Score_MEWS' successfully added
Variable 'add_triage_MAP' successfully added
Variable 'add_score_CCI' successfully added
Variable 'Score_CART' successfully added
Variable 'Score_REMS' successfully added
Variable 'Score_NEWS' successfully added
Variable 'Score_NEWS2' successfully added
Variable 'Score_MEWS' successfully added


### 6. Output the train and test data

In [33]:
df_train.to_csv(os.path.join(path, 'train.csv'), index=False)
df_validation.to_csv(os.path.join(path, 'validation.csv'), index=False)
df_test.to_csv(os.path.join(path, 'test.csv'), index=False)

## Time series data processing

In [34]:
#mimic_iv_ed_path = 'your_path/mimic-iv-ed-1.0/ed'
df_vitalsign = read_vitalsign_table(os.path.join(mimic_iv_path, 'ed/vitalsign.csv'))

In [35]:
df_vitalsign['charttime'] = pd.to_datetime(df_vitalsign['charttime'])
df_vitalsign.sort_values('charttime', inplace=True)
df_vitalsign.drop('ed_rhythm', axis=1, inplace=True)
df_vitalsign.head()

,subject_id,stay_id,charttime,ed_temperature,ed_heartrate,ed_resprate,ed_o2sat,ed_sbp,ed_dbp,ed_pain
52797,13238787,35341790,2110-01-11 01:49:00,98.4,77.0,16.0,100.0,149.0,104.0,8.0
1417298,15350437,39042378,2110-01-11 03:45:00,97.1,71.0,16.0,100.0,117.0,79.0,0.0
52798,13238787,35341790,2110-01-11 04:02:00,98.0,78.0,18.0,99.0,138.0,92.0,0.0
1417299,15350437,39042378,2110-01-11 05:42:00,98.4,57.0,16.0,100.0,132.0,90.0,0.0
1417300,15350437,39042378,2110-01-11 08:10:00,99.1,83.0,18.0,100.0,152.0,97.0,0.0


In [36]:
grouped = df_vitalsign.groupby('stay_id')

In [37]:
resample_freq = '1H' # 1 hour
# resample_freq = '30T' # 30 minutes

df_list = []
counter = 0
N = len(grouped)
for stay_id, stay_df in grouped:
    counter += 1
    stay_df.set_index('charttime', inplace=True)
    stay_df = stay_df.resample('1T', origin='start').interpolate(method='linear')\
                     .resample(resample_freq, origin='start').asfreq().ffill().bfill()
    if counter % 10000 == 0 or counter == N:
        print('%d/%d' % (counter, N), end='\r')
    df_list.append(stay_df)

In [44]:
df_vitalsign_resampled = pd.concat(df_list)

In [45]:
df_vitalsign_resampled = convert_temp_to_celcius(df_vitalsign_resampled)

In [46]:
df_vitalsign_resampled = remove_outliers(df_vitalsign_resampled, vitals_valid_range)

In [47]:
vitals_cols = [col for col in df_vitalsign_resampled.columns if len(col.split('_')) > 1 and 
                                                                col.split('_')[1] in vitals_valid_range]
vitals_cols

['ed_temperature',
 'ed_heartrate',
 'ed_resprate',
 'ed_o2sat',
 'ed_sbp',
 'ed_dbp',
 'ed_pain']

In [48]:
imputer = SimpleImputer(strategy='median')
df_vitalsign_resampled[vitals_cols] = imputer.fit_transform(df_vitalsign_resampled[vitals_cols])

In [49]:
df_vitalsign_resampled.to_csv(os.path.join(output_path, 'ed_vitalsign_'+resample_freq+'_resampled.csv'))